In [3]:
import os
import yaml
import psycopg2
import pandas as pd
import urllib.request
from sqlalchemy import text
from sqlalchemy import create_engine
import time
import random
import numpy as np
import scipy.sparse as sparse

In [4]:
# setting WD
os.chdir('/Users/Daniel.Kwon/projects')

In [5]:
# database connection info
with open("./redshift_credentials.yml", 'r') as stream:
    try:
        rs_cred = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
engine_url = "postgresql://{user_name}:{pw}@{host}:{port}/{db_name}".format(user_name = rs_cred['default']['user'],
                                                                            pw        = urllib.parse.quote_plus(rs_cred['default']['password']),
                                                                            host      = rs_cred['default']['host'],
                                                                            port      = rs_cred['default']['port'],
                                                                            db_name   = rs_cred['default']['dbname']
                                                                           )
engine = create_engine(engine_url)

In [6]:
# sessions
query_1 = open('./recommender-systems/notebooks/user-item-viewcount.sql', 'r')
query_text_1 = text(query_1.read())
sessions = pd.read_sql_query(query_text_1, engine)
# products
query_2 = open('./recommender-systems/notebooks/items.sql', 'r')
query_text_2 = text(query_2.read())
products = pd.read_sql_query(query_text_2, engine)

In [68]:
sessions.head()

,ip_address,user_id,product_id,total_views
0,44.227.101.58,323063031612,4578206875703,7793
1,44.227.101.58,13364489851595,4472283758647,880
2,52.24.27.166,14512521876382,4578206875703,702
3,44.227.101.58,14652643828321,1451827101751,518
4,44.227.101.58,16010249681529,4417468039223,274


In [56]:
# creating a table with "confidences"
data = sessions.groupby(['user_id', 'product_id'])[['total_views']].sum().reset_index()
data.columns=["user_id", "product_id", "total_views"]
data.product_id = data.product_id.astype('int64')

In [ ]:
data.head()

In [73]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(data.product_id.unique()))
views = list(data.total_views)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((views, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

ValueError: row index exceeds matrix dimensions

In [ ]:
products_rows.max()

In [66]:
len(items) + 1

901

In [67]:
products_rows[0:5]

0    4578206875703
1    1575988953143
2    4578206416951
3    4578206548023
4    4578206875703
Name: product_id, dtype: int64